In [237]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import scipy
import scipy.io
import matplotlib.pyplot as plt
import random
import pickle
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
from scipy.stats import pearsonr
from utils import *
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

In [238]:
device = torch.device(0)
device

device(type='cuda', index=0)

In [239]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1 = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2 = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3 = np.delete(data_glove_3, 3, 1)

ecog_1 = raw['train_ecog'][0][0]
ecog_2 = raw['train_ecog'][1][0]
ecog_3 = raw['train_ecog'][2][0]

labels_1 = np.argmax(data_glove_1, axis=1)
labels_2 = np.argmax(data_glove_2, axis=1)
labels_3 = np.argmax(data_glove_3, axis=1)

In [240]:
ecog_1 = filter_data(ecog_1)
ecog_2 = filter_data(ecog_2)
ecog_3 = filter_data(ecog_3)

train_test_ratio = 0.7

ecog_1_train = ecog_1[:int(train_test_ratio * ecog_1.shape[0])]
ecog_1_test = ecog_1[int(train_test_ratio * ecog_1.shape[0]):]
data_glove_1_train = data_glove_1[:int(train_test_ratio * data_glove_1.shape[0])]
data_glove_1_test = data_glove_1[int(train_test_ratio * data_glove_1.shape[0]):]

ecog_2_train = ecog_2[:int(train_test_ratio * ecog_2.shape[0])]
ecog_2_test = ecog_2[int(train_test_ratio * ecog_2.shape[0]):]
data_glove_2_train = data_glove_2[:int(train_test_ratio * data_glove_2.shape[0])]
data_glove_2_test = data_glove_2[int(train_test_ratio * data_glove_2.shape[0]):]

ecog_3_train = ecog_3[:int(train_test_ratio * ecog_3.shape[0])]
ecog_3_test = ecog_3[int(train_test_ratio * ecog_3.shape[0]):]
data_glove_3_train = data_glove_3[:int(train_test_ratio * data_glove_3.shape[0])]
data_glove_3_test = data_glove_3[int(train_test_ratio * data_glove_3.shape[0]):]

In [395]:
def NumWins(x, fs, winLen, winDisp):
    return int(1 + (x.shape[0] - winLen * fs) / (winDisp * fs))

winLen = 200 / 1e3
winOverlap = 40 / 1e3
winDisp = winLen - winOverlap
NumWins(ecog_1, 1000, winLen, winDisp)

1874

In [398]:
def LineLength(x):
    return np.abs(np.diff(x, axis=0)).sum(axis=0)

def Area(x):
    return np.abs(x).sum(axis=0)

def Energy(x):
    return (x ** 2).sum(axis=0)

def ZeroCrossingMean(x):
    return ((x < x.mean(axis=0))[1:] & (x[:-1] > x.mean(axis=0)) | (x > x.mean(axis=0))[1:] & (x[:-1] < x.mean(axis=0))).sum(axis=0)

def numSpikes(x):
    #TODO: implement
    sig.find_peaks(x, height=0, distance=100)
    pass

def averageTimeDomain(x):
    #TODO: implement
    return np.mean(x, axis=0)

def bandpower(x, fs, fmin, fmax):
    fs = 1000
    # win = 4 * sf
    freqs, psd = sig.welch(x, fs, axis=0, nperseg=x.shape[0])
    
    # Define delta lower and upper limits
    # fmin, fmax = 0.5, 4

    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs >= fmin, freqs <= fmax)
    
    from scipy.integrate import simps

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    delta_power = simps(psd[idx_delta], dx=freq_res, axis=0)
    
    return delta_power

def spectral_entropy(x, fs=1000):
    # Calculate the power spectrum
    f, Pxx = sig.welch(x, fs=fs)
    # Normalize the power spectrum
    Pxx_norm = Pxx / Pxx.sum()
    # Calculate the spectral entropy
    se = -1 * (Pxx_norm * np.log2(Pxx_norm)).sum()
    return se

def hjorth_complexity(x):
    dx = np.diff(x)
    d2x = np.diff(dx)
    var_x = np.var(x)
    var_dx = np.var(dx)
    var_d2x = np.var(d2x)
    activity = var_x
    mobility = np.sqrt(var_d2x / var_dx)
    # Calculate Hjorth complexity
    complexity = mobility / activity
    return complexity
    
# Kurtosis = @(x) ((1/size(x,1))*sum((x - mean(x)).^4))./(((1/size(x,1))*sum((x - mean(x)).^2)).^2);
def Kurtosis(x):
    return ((1/x.shape[0])*np.sum((x - np.mean(x))**4))/(((1/x.shape[0])*np.sum((x - np.mean(x))**2))**2)

def get_features(filtered_window, fs=1000):
    """
        Write a function that calculates features for a given filtered window. 
        Feel free to use features you have seen before in this class, features that
        have been used in the literature, or design your own!

        Input: 
        filtered_window (window_samples x channels): the window of the filtered ecog signal 
        fs: sampling rate
        Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    feat_LL = LineLength(filtered_window)
    feat_Area = Area(filtered_window)
    feat_Energy = Energy(filtered_window)
    feat_ZCM = ZeroCrossingMean(filtered_window)
    feat_TimeAvg = averageTimeDomain(filtered_window)
    feat_SpectralEntropy = spectral_entropy(filtered_window)
    feat_Hijorth = hjorth_complexity(filtered_window)
    feat_kurtosis = Kurtosis(filtered_window)
    # feat_FreqAvg = averageFreqDomain(filtered_window)
    
    from pyriemann.estimation import Covariances
    from pyriemann.tangentspace import TangentSpace
    
    # covar = Covariances().fit_transform(np.expand_dims(filtered_window.T, 0))
    # # covest = Covariances('oas')
    # # temp = np.expand_dims(filtered_window, axis=-1)
    # # covar = covest.fit_transform(temp)
    # ts = TangentSpace()
    # tsfeat = ts.fit_transform(covar)
    # # print(tsfeat.shape)

    # raise notImplementedError()
    return np.hstack([#feat_LL, feat_Area, feat_Energy, feat_ZCM, 
                      feat_TimeAvg, 
                      #feat_SpectralEntropy,
                      feat_Hijorth,
                      feat_kurtosis,
                      bandpower(filtered_window, 1000, 5, 15),
                      bandpower(filtered_window, 1000, 20, 25),
                      bandpower(filtered_window, 1000, 75, 115),
                      bandpower(filtered_window, 1000, 125, 160),
                      bandpower(filtered_window, 1000, 160, 175)])

In [399]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    """
        Write a function which processes data through the steps of filtering and
        feature calculation and returns features. Points will be awarded for completing
        each step appropriately (note that if one of the functions you call within this script
        returns a bad output, you won't be double penalized). Note that you will need
        to run the filter_data and get_features functions within this function. 

        Inputs:
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length: the window's length
        window_overlap: the window's overlap
        Output: 
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
            note that this is a 2D array. 
    """
    raw_ecog = filter_data(raw_ecog, fs)
    
    window_disp = window_length - window_overlap
    
    all_feats = np.vstack([get_features(raw_ecog[int(i * window_disp * fs):int(i * window_disp * fs + window_length * fs), :], fs) for i in range(NumWins(raw_ecog, fs, window_length, window_disp
))])
    
    return all_feats

In [439]:
temp = ecog_1[:500]
convar = np.cov(temp, rowvar=False)
convar[0, :1]

array([5013630.20390603])

In [400]:
def create_R_matrix(features, N_wind):
    """ 
    Write a function to calculate the R matrix

    Input:
        features (samples (number of windows in the signal) x channels x features): 
        the features you calculated using get_windowed_feats
        N_wind: number of windows to use in the R matrix

    Output:
        R (samples x (N_wind*channels*features))
    """
    num_win = features.shape[0]
    num_channel_features = features.shape[1]
    
    # Append a copy of the first N-1 rows to the beginning of features
    features = np.vstack((features[:N_wind-1], features))
    
    R = np.zeros((num_win, N_wind * num_channel_features))
    
    for i in range(num_win):
        # Get the feature matrix for the current window
        # Resize the feature matrix and store in R
        R[i,:] = features[i:i+N_wind,:].reshape(-1)

    R = np.hstack((np.ones((R.shape[0], 1)), R))

    return R
    

In [417]:
winLen = 200 / 1e3
winOverlap = 40 / 1e3
winDisp = winLen - winOverlap

feature_train = get_windowed_feats(ecog_1_train, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 20)

feature_test = get_windowed_feats(ecog_1_test, 1000, winLen, winOverlap)
R_test = create_R_matrix(feature_test, 20)

y_train = data_glove_1_train
y_test = data_glove_1_test
y_train = sig.resample(y_train, R_train.shape[0], axis=0)
y_test = sig.resample(y_test, R_test.shape[0], axis=0)

In [418]:
R_test.shape

(562, 7481)

In [419]:
class FingerFeatureDataset(Dataset):
    def __init__(self, ecog, dg, window=2000):
        self.ecog = np.float32(ecog + np.random.normal(0, 0.01, ecog.shape))
        self.ecog = (self.ecog - self.ecog.mean(axis=0)) / self.ecog.std(axis=0)
#         self.ecog = self.ecog.reshape(self.ecog.shape[0], 1, -1)
        self.dg = np.float32(dg)
        
    

    def __len__(self):
        return len(self.ecog)

    def __getitem__(self, idx):
        
        return self.ecog[idx], self.dg[idx]

In [420]:
dataset_s1_train = FingerFeatureDataset(R_train.copy(), y_train.copy())
dataset_s1_valid = FingerFeatureDataset(R_test.copy(), y_test.copy())

train_loader = DataLoader(dataset_s1_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s1_valid, batch_size=16, shuffle=False)

In [421]:
class FingerRegressor(nn.Module):
    def __init__(self, num_fingers) -> None:
        super(FingerRegressor, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 61, stride=4)
        self.pool1 = nn.AvgPool1d(16, 16)
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(7481, 64)
        self.fc2 = nn.Linear(64, 16)
#         self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(16, num_fingers)
        
        self.relu = nn.ReLU()
        
        nn.init.xavier_normal_(self.fc1.weight)
#         nn.init.xavier_normal_(self.fc2.weight)
#         nn.init.xavier_normal_(self.fc3.weight)
        nn.init.xavier_normal_(self.fc4.weight)
        self.dropout1 = nn.Dropout(0.8)
        self.dropout2 = nn.Dropout(0.4)
        
    def forward(self, x):
        x = self.dropout1(x)
#         x = self.relu(self.conv1(x))
#         x = self.pool1(x)
        
#         x= self.dropout2(x)
        
#         x = self.flatten(x)
        
        x = self.relu(self.fc1(x))
        
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
#         x = self.relu(self.fc3(x))
        
        output = self.fc4(x)
        
        return output

In [422]:
# net = EEGNet().to(device)
# net = EEGNetRegressor(4).to(device)
# net = EEGNet(n_classes=4, channels=62, samples=3000).to(device) #.cuda(0)
net = FingerRegressor(4).to(device)
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))#.cuda(0))))
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.1)


for epoch in range(50):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_1_train)), data_glove_1_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(data_glove_1_test)), data_glove_1_test)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 2.378 | Train Cor: -0.016 | Valid loss: 0.827 | Valid Cor: 0.001615105653582246
Epoch 2 | Train loss: 1.129 | Train Cor: 0.018 | Valid loss: 0.813 | Valid Cor: 0.03079252470395183
Epoch 3 | Train loss: 1.038 | Train Cor: 0.050 | Valid loss: 0.810 | Valid Cor: 0.027552631337388188
Epoch 4 | Train loss: 1.003 | Train Cor: 0.081 | Valid loss: 0.808 | Valid Cor: 0.05312594004161772
Epoch 5 | Train loss: 0.997 | Train Cor: 0.078 | Valid loss: 0.808 | Valid Cor: 0.06655137603871591
Epoch 6 | Train loss: 0.988 | Train Cor: 0.112 | Valid loss: 0.804 | Valid Cor: 0.10800499369773034
Epoch 7 | Train loss: 0.972 | Train Cor: 0.173 | Valid loss: 0.800 | Valid Cor: 0.11572320596536767
Epoch 8 | Train loss: 0.973 | Train Cor: 0.167 | Valid loss: 0.797 | Valid Cor: 0.12635799301394215
Epoch 9 | Train loss: 0.961 | Train Cor: 0.199 | Valid loss: 0.795 | Valid Cor: 0.14112172741615106
Epoch 10 | Train loss: 0.949 | Train Cor: 0.237 | Valid loss: 0.790 | Valid Cor: 0.15781378460080

In [424]:
correlation(pred, y_test)

([0.3327978217891593,
  0.39574460065139055,
  0.019683200175373804,
  0.2680101301775504],
 0.25405893819836856)